In [ ]:
import platform

arch = platform.machine()
sys = platform.system()
processor = platform.processor()
print(f"{arch}\n{sys}\n{processor}")

In [ ]:
import onnxruntime as ort
import os
import numpy as np
import time
import gc
import psutil

from pathlib import Path
from tokenizers import Tokenizer

In [ ]:
root_dir = Path.cwd().parent.parent
onnx_root = Path(ort.__file__).parent

In [ ]:
root_dir

In [ ]:
onnx_root

In [ ]:
# Subdirectory where all .onnx dependencies are located
model_subdirectory = "qnn-deepseek-r1-distill-qwen-7b"

# The embeddings model is entry point, use netron to visualize
model_name = "deepseek_r1_7b_embeddings_quant_v1.0.onnx"

# This graph is used to process initial prompt, we can pass up to 64 tokens
context_model = "deepseek_r1_7b_ctx_v1.0.onnx_ctx.onnx"

# This graph is used to perform next word inference after the initial prompt
context_model_iter = "deepseek_r1_7b_iter_v1.0.onnx_ctx.onnx"

# This graph allows us to take hidden states and return logits
head_model = "deepseek_r1_7b_head_quant_v1.0.onnx"

# Tokenizer
tokenizer_json = "tokenizer.json"

In [ ]:
model_path = root_dir/"models"/model_subdirectory/model_name
ctx_path = root_dir/"models"/model_subdirectory/context_model
ctx_path_itr = root_dir/"models"/model_subdirectory/context_model_iter
head_path = root_dir/"models"/model_subdirectory/head_model
tokenizer_path = root_dir/"models"/model_subdirectory/tokenizer_json
hexagon_driver = onnx_root/"capi"/"QnnHtp.dll"

In [ ]:
hexagon_driver.is_file()

In [ ]:
session_options = ort.SessionOptions()

qnn_provider_options = {
    # Path to the backend driver "Hexagon"
    "backend_path": hexagon_driver,
    # https://onnxruntime.ai/docs/execution-providers/QNN-ExecutionProvider.html#configuration-options
    "htp_performance_mode": "burst",
    "soc_model": "60",
    # "enable_htp_context_cache": "0",
    # "profiling_level": "detailed",
    # "profiling_file_path": root_dir/"models"/model_subdirectory/"profiling_deepseek_7b.csv",
    # Enabling graph optimization causes problems, need to look into this
    "htp_graph_finalization_optimization_mode": "3",
    "qnn_context_priority":"high",
}

In [ ]:
embedding_session = ort.InferenceSession(model_path,
                                providers= [("QNNExecutionProvider",qnn_provider_options)],
                               sess_options= session_options
                              )

In [ ]:
# Creating an inference session for the initial context graph
ctx_session = ort.InferenceSession(ctx_path,
                                    providers=[("QNNExecutionProvider",qnn_provider_options)],
                                    sess_options= session_options
                                        )

In [ ]:
# Creating an inference session for the single prediction context graph (iter_ctx)
ctx_itr_session = ort.InferenceSession(ctx_path_itr,
                                         providers=[("QNNExecutionProvider",qnn_provider_options)],
                                         sess_options= session_options
                                      )

In [ ]:
# Creating an inference session for the head session which will provide logits from hidden states
head_session = ort.InferenceSession(head_path,
                                providers= [("QNNExecutionProvider",qnn_provider_options)],
                               sess_options= session_options
                              )

In [ ]:
ctx_session.get_providers()

In [ ]:
tokenizer = Tokenizer.from_file(str(tokenizer_path))

In [ ]:
query = "<｜User｜>\nImagine you are a cyber security professional. Provide step by step reasons why AI models should be ran locally. Please consider all aspects of data privacy and cyber security\n<｜Assistant｜><think>\n"
query

In [ ]:
encoding = tokenizer.encode(query)
encoding

In [ ]:
encoding.ids

In [ ]:
len(encoding.ids)

In [ ]:
for layer in embedding_session.get_inputs():
    print(f"Name: {layer.name}")
    print(f"Shape: {layer.shape}")
    print(f"Type: {layer.type}")

In [ ]:
input_ids = encoding.ids
input_ids

In [ ]:
input_ids = np.array([input_ids], dtype=np.int64)
input_ids.shape

In [ ]:
embedding_output = embedding_session.run(None, {"input_ids": input_ids})[0]
print("(batch, sequence length, embedding dimension)")
embedding_output.shape

In [ ]:
for layer in embedding_session.get_outputs():
    print(f"Name: {layer.name}")
    print(f"Shape: {layer.shape}")
    print(f"Type: {layer.type}")

In [ ]:
# Number of input sequences processed simultaneously
batch_size = 1

# Current sequence length for initial prompt (number of tokens in current sequence)
seq_len = embedding_output.shape[1]

# Dimensionality of each token embedding vector
hidden_size = embedding_output.shape[2]

# Number of attention heads in each transformer layer
num_heads = 28

# Size of each attention head (should be hidden_size // num_heads
attn_head_size = 128 

# Total number of transformer layers
num_layers = 28

# SWA
max_seq_len = 64

# Number of key/value heads (key/value heads are shared amongst attention heads)
num_key_value_heads = 4

In [ ]:
for layers in ctx_session.get_inputs():
    print(f"Name: {layers.name}")
    print(f"Shape: {layers.shape}")
    print(f"Type: {layers.type}")

In [ ]:
# Let's initialize our KV cache for all transformer layers
empty_kv = {}
for i in range(num_layers):
    # Shape of key and value tensors for each transformer layer
    past_shape = (batch_size, num_key_value_heads, max_seq_len, attn_head_size)

    # Initialize past keys for layer i (used in attention mechanism to avoid recomputation
    empty_kv[f"past_keys_{i}"] = np.zeros(past_shape, dtype=np.float32)

    # Initialize past values for layer i
    empty_kv[f"past_values_{i}"] = np.zeros(past_shape, dtype=np.float32)

len(empty_kv)

In [ ]:
empty_kv.keys()

In [ ]:
# Subtract 1 to get the index of the last token in the sequence (since indexing is 0-based)
init_sequence_length = np.array(embedding_output.shape[1]-1, dtype=np.int32).reshape(1,1)

# Set the maximum sequence length for the model's current forward pass
max_seq_length = np.array([max_seq_len], dtype=np.int32)

In [ ]:
seq_lens = {
    "past_seq_len": init_sequence_length,
    "total_seq_len": max_seq_length 
}
seq_lens

In [ ]:
init_sequence_length.shape

In [ ]:
max_seq_length.shape

In [ ]:
print(tokenizer.token_to_id("<｜end▁of▁sentence｜>"))

In [ ]:
# pad the inputs to expected size of prefill graph
batch_size, seq_len, embed_dim = embedding_output.shape
padding_id = 151643
padded_embedding = np.full((batch_size, max_seq_length[0], embed_dim), padding_id, dtype=embedding_output.dtype)
padded_embedding[:, :seq_len, :] = embedding_output 
padded_embedding.shape

In [ ]:
padded_embedding[:, :seq_len+1, :]

In [ ]:
embedding_output[:, -1, :]

In [ ]:
prefill_inputs = {
    **empty_kv,
    **seq_lens,
    "input_hidden_states": padded_embedding,
}
prefill_inputs

In [ ]:
prompt_outputs = ctx_session.run(None, prefill_inputs)
len(prompt_outputs)

In [ ]:
for layer in ctx_session.get_outputs():
    print(f"Name: {layer.name}")
    print(f"Shape: {layer.shape}")
    print(f"Type: {layer.type}")

In [ ]:
print("Batch, key/value heads, prompt length, head dimension (size of projection for each head)")
prompt_outputs[1].shape

In [ ]:
# Extract final hidden states and present_keys/values
print("Batch, prompt length, vector embedding size")
output_hidden_states = prompt_outputs[0]
output_hidden_states.shape

In [ ]:
prompt_outputs[1 + 0 * 2]

In [ ]:
# Update KV cache from Prefill calculation
# Must start with index==1 because index==0 is output_hidden_states
present_kv = {f"past_keys_{i}": prompt_outputs[1 + i * 2] for i in range(num_layers)}
present_kv.update({f"past_values_{i}": prompt_outputs[1 + i * 2 + 1] for i in range(num_layers)})
present_kv

In [ ]:
for layer in head_session.get_inputs():
    print(f"Name: {layer.name}")
    print(f"Shape: {layer.shape}")
    print(f"Type: {layer.type}")

In [ ]:
logits = head_session.run(None, {"output_hidden_states": output_hidden_states})[0]
logits

In [ ]:
for layer in head_session.get_outputs():
    print(f"Name: {layer.name}")
    print(f"Shape: {layer.shape}")
    print(f"Type: {layer.type}")

In [ ]:
print("Batch Size, Sequence Length, Vocabulary Size")
logits.shape

In [ ]:
def softmax_numpy(x: np.array, temperature: float=1) -> np.array:
    # stabilize x in case of large numbers 
    x = x - np.max(x, axis=-1, keepdims=True)

    # Apply temperature
    x = x/temperature

    # Apply Softmax
    return np.exp(x)/np.sum(np.exp(x), axis=-1, keepdims=True)

In [ ]:
def top_k_probas(probas: np.array, k: int=5) -> np.array:
    # Copy probas so in-place operations don't work on original variable
    probas = probas.copy()
    # Normalize probabilities
    probas /= np.sum(probas)
    # Using -probas to get in descending order
    top_indices_sorted = np.argsort(-probas)[:k]
    top_k_probas = probas[top_indices_sorted]

    # Renormalize top-k probabilites to sum to 1 (probabilites must sum to 1 to use np.random.choice
    top_k_probas /= np.sum(top_k_probas)

    # Return top k probabilities
    return top_indices_sorted, top_k_probas

In [ ]:
def apply_repetition_penalty(logits, generated_ids, penalty=1.1):
    for token_id in set(generated_ids):
        logits[token_id] /= penalty
    return logits

In [ ]:
softmax = lambda x, temperature=1: np.exp((x-np.max(x, axis=-1, keepdims=True))/temperature)/np.sum(np.exp((x-np.max(x, axis=-1, keepdims=True))/temperature), axis=-1, keepdims=True)

In [ ]:
# Sampling temperature for softmax-based logit scaling
temp = 0.7
probas = softmax(logits[0,-1], temperature=temp)

next_token_id = int(np.random.choice(len(probas), p=probas))

In [ ]:
tokenizer.decode([next_token_id])

In [ ]:
for layer in ctx_itr_session.get_inputs():
    print(f"Name: {layer.name}")
    print(f"Shape: {layer.shape}")
    print(f"Type: {layer.type}")

In [ ]:
start = time.time()
max_tokens = 100
top_k = 5
generated_ids = [next_token_id]
prev_seq_len = 64

print("\nInitial Query:\n", query)
print("Generated:")

for _ in range(max_tokens):
    input_ids = np.array([[next_token_id]], dtype=np.int64)
    print(tokenizer.decode([next_token_id], skip_special_tokens=True),end="")
    embedding_output = embedding_session.run(None, {"input_ids": input_ids})[0]
    
    lengths = {
    "past_seq_len": np.array([[prev_seq_len]], dtype=np.int32),
    "total_seq_len": np.array([prev_seq_len + 1], dtype=np.int32)
    }

    iter_inputs = {
    "input_hidden_states": embedding_output,
    **present_kv,
    **lengths,
    }

    iter_outputs = ctx_itr_session.run(None, iter_inputs)

    output_hidden_states = iter_outputs[0]

    present_kv = {f"past_keys_{i}": iter_outputs[1 + i * 2] for i in range(num_layers)}
    present_kv.update({f"past_values_{i}":iter_outputs[1 + i * 2 + 1] for i in range(num_layers)})
    
    logits = head_session.run(None, {"output_hidden_states": output_hidden_states})[0]    

    token_logits = logits[0,-1]
    token_logits = apply_repetition_penalty(token_logits, generated_ids, penalty=1.1)
    # Get probabilities
    probas = softmax(token_logits, temperature=temp)
    top_indices, top_probas = top_k_probas(probas, k=top_k) 
    next_token_id = int(np.random.choice(top_indices, p=top_probas)) #int(np.argmax(probas))
    generated_ids.append(next_token_id)
    prev_seq_len += 1

    if next_token_id == tokenizer.token_to_id("<｜end▁of▁sentence｜>"):
        break
        
end = time.time()
elapsed = end - start
tps = np.round((max_tokens / elapsed), 2)
print(f"\nTokens Per Second: {tps}")
output_text = tokenizer.decode(generated_ids, skip_special_tokens=True)